### Data

Downloaded from: https://www.kaggle.com/zynicide/wine-reviews

### Question 1.

Summary of data set: A row represents information about a bottle of wine including country where it is produced, description written by a sommelier, score points rated in a wine review site, price of a bottle and etc.

Essential columns:
* country
* description

Suppose that we want to discover the unique words used in the description of wines produced in a single country. Since the words used rarely in the descriptions are, however, usually uninformative, we are going to count those frequently occur in wine descriptions of each country.

Our strategy to analyze the wine data set is as follows: 
(1) Count how many times each word appears in the descriptions about the wine bottles produced in each country.
(2) For each country, calculate the set of top-100 frequent words shown in descriptions. 
(3) Then, find the words that appear in the "top-100 frequent word set" of a single country.

### 문제 1.

데이터 셋 요약: 각 행의 데이터는 와인에 대한 정보, 즉, 생산국, 소믈리에가 작성한 설명문, 와인 리뷰 사이트에 따른 평점, 병 당 가격 등의 정보를 담고있다.

본 문제에 사용되는 핵심 컬럼은 다음과 같다.
* country
* decription

와인 설명 글에 사용되는 단어 중에서 오직 한 나라의 와인들을 설명하기 위해서만 사용되는 독특한 단어를 찾고 싶다고 생각해보자. 그러나 나라별로 흔히 사용되지 않는 단어를 제외하고 여러번 등장하는 단어들 사이에서 그러한 독특한 단어를 찾기위해서 나라별로 빈번히 나타나는 단어들을 뽑아서 그중에 오직 한나라의 와인 설명에만 사용되는 단어를 뽑고자 한다.

아래 코드에서 데이터를 분석하는 과정은 크게 다음과 같다: (1) 각 나라별로 와인 설명문에 등장하는 각 단어들의 출현 빈도를 센다. (2) 각 나라별로 출현빈도수 에 따라 순위 100개의 단어를 뽑는다. (3) 오직 한 나라의 출현빈도수 100위안에 드는 단어를 뽑는다.

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 39 kB/s 
     |████████████████████████████████| 199 kB 58.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=d3283599045b8edece8a39df0613746d61932578a018d47ebf7ae05fc9cefc08
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [1]:
# create spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("my app").master("local").getOrCreate()

# get context from the session
sc = spark.sparkContext

22/11/17 08:18:49 WARN Utils: Your hostname, cherry resolves to a loopback address: 127.0.1.1; using 192.168.0.7 instead (on interface enp6s0)
22/11/17 08:18:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/17 08:18:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/17 08:18:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


(1) 우선 winemag-data-130k-v2.csv 파일을 읽어 컬럼 "country", "description" 만을 남겨 df란 이름의 데이터프레임에 저장하시오 (5점)

In [45]:
data = spark.read.format('csv').option("sep", ",").option('header', 'true').load('./Wine_review/winemag-data-130k-v2.csv')
df = data.select('Country', 'description')

In [3]:
df.show()

+---------+--------------------+
|  Country|         description|
+---------+--------------------+
|    Italy|Aromas include tr...|
| Portugal|This is ripe and ...|
|       US|Tart and snappy, ...|
|       US|Pineapple rind, l...|
|       US|Much like the reg...|
|    Spain|Blackberry and ra...|
|    Italy|Here's a bright, ...|
|   France|This dry and rest...|
|  Germany|Savory dried thym...|
|   France|This has great de...|
|       US|Soft, supple plum...|
|   France|This is a dry win...|
|       US|Slightly reduced,...|
|    Italy|This is dominated...|
|       US|Building on 150 y...|
|  Germany|Zesty orange peel...|
|Argentina|Baked plum, molas...|
|Argentina|Raw black-cherry ...|
|    Spain|Desiccated blackb...|
|       US|Red fruit aromas ...|
+---------+--------------------+
only showing top 20 rows



(2) df에 ***description이 None이거나 country가 None인*** row가 있는지 확인하고 (즉, 그러한 row의 개수를 세어보고 출력해보고고) 있으면 (즉, 둘중에 하나라도 None인 row가 있다면) filter를 이용해 이를 제거하여 그 결과의 RDD를 rdd1이란 이름의 변수에 저장하시오 (10점)

In [46]:
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()

+-------+-----------+
|Country|description|
+-------+-----------+
|     63|          1|
+-------+-----------+



In [47]:
rdd1 = df.dropna(subset=["Country"])
rdd1 = rdd1.dropna(subset=["description"])
rdd1.show()

+---------+--------------------+
|  Country|         description|
+---------+--------------------+
|    Italy|Aromas include tr...|
| Portugal|This is ripe and ...|
|       US|Tart and snappy, ...|
|       US|Pineapple rind, l...|
|       US|Much like the reg...|
|    Spain|Blackberry and ra...|
|    Italy|Here's a bright, ...|
|   France|This dry and rest...|
|  Germany|Savory dried thym...|
|   France|This has great de...|
|       US|Soft, supple plum...|
|   France|This is a dry win...|
|       US|Slightly reduced,...|
|    Italy|This is dominated...|
|       US|Building on 150 y...|
|  Germany|Zesty orange peel...|
|Argentina|Baked plum, molas...|
|Argentina|Raw black-cherry ...|
|    Spain|Desiccated blackb...|
|       US|Red fruit aromas ...|
+---------+--------------------+
only showing top 20 rows



(3) rdd에 있는 각 Row의 description을 처리하여 나라별 단어의 출연횟수를 카운트하기 위해 적절한 flatMap을 이용한 transformation을 수행하시오. 즉, description을 띄어쓰기 (" ") 단위로 잘라서 각 단어마다 country 와 단어의 쌍(tuple)을 키로하고 값을 1로 하는 키-값 쌍을 생성하도록 flatMap을 작성하시오. 더불어, 단어는 대소문자를 구분하지 않기 위해 모두 소문자로 바꾸어 두시오. flatMap 수행 결과 RDD는 rdd2 란 이름의 변수에 저장하시오. (10점)

In [48]:
dftordd = rdd1.rdd.map(lambda x: [x[0].lower(), x[1].lower().split(' ')])

In [49]:
rdd2 = []

for word in dftordd.collect():
    for j in range(len(word[1])):
        rdd2.append((word[0], word[1][j]))

rdd2 = sc.parallelize(rdd2)

(4) rdd2의 키 (즉, country와 단어의 쌍의 tuple)를 카운트하는 transformation을 작성하시오. 그 결과는 rdd3에 저장하시오 (10점)

In [50]:
rdd3 = rdd2.map(lambda x: (x,1)).reduceByKey(lambda x, y: x+y)

In [51]:
rdd3.collect()

22/11/17 08:50:59 WARN TaskSetManager: Stage 55 contains a task of very large size (75002 KiB). The maximum recommended task size is 1000 KiB.


[(('italy', 'aromas'), 11805),
 (('italy', 'include'), 279),
 (('italy', 'tropical'), 316),
 (('italy', 'fruit,'), 2203),
 (('italy', 'broom,'), 107),
 (('italy', 'brimstone'), 41),
 (('italy', 'and'), 52389),
 (('italy', 'dried'), 3036),
 (('italy', 'herb.'), 351),
 (('italy', 'the'), 32393),
 (('italy', 'palate'), 9330),
 (('italy', "isn't"), 121),
 (('italy', 'overly'), 22),
 (('italy', 'expressive,'), 71),
 (('italy', 'offering'), 419),
 (('italy', 'unripened'), 3),
 (('italy', 'apple,'), 937),
 (('italy', 'citrus'), 1449),
 (('italy', 'sage'), 687),
 (('italy', 'alongside'), 4494),
 (('italy', 'brisk'), 281),
 (('italy', 'acidity.'), 2071),
 (('portugal', 'this'), 5504),
 (('portugal', 'is'), 7475),
 (('portugal', 'ripe'), 1831),
 (('portugal', 'and'), 11689),
 (('portugal', 'fruity,'), 307),
 (('portugal', 'a'), 8897),
 (('portugal', 'wine'), 5204),
 (('portugal', 'that'), 2531),
 (('portugal', 'smooth'), 407),
 (('portugal', 'while'), 519),
 (('portugal', 'still'), 650),
 (('por

(5) groupByKey를 이용하여 나라별로 나타나는 단어와 그 count 정보를 모으고자 한다. 우선은 rdd3을 transformation하여 country를 key로 하고 단어 및 단어의 count의 튜플을 value로하는 RDD를 생성하여 rdd4에 저장하자 (10점) 

In [52]:
rdd4 = rdd3.map(lambda x:(x[0][0], (x[0][1], x[1])))
rdd4.collect()

[('italy', ('aromas', 11805)),
 ('italy', ('include', 279)),
 ('italy', ('tropical', 316)),
 ('italy', ('fruit,', 2203)),
 ('italy', ('broom,', 107)),
 ('italy', ('brimstone', 41)),
 ('italy', ('and', 52389)),
 ('italy', ('dried', 3036)),
 ('italy', ('herb.', 351)),
 ('italy', ('the', 32393)),
 ('italy', ('palate', 9330)),
 ('italy', ("isn't", 121)),
 ('italy', ('overly', 22)),
 ('italy', ('expressive,', 71)),
 ('italy', ('offering', 419)),
 ('italy', ('unripened', 3)),
 ('italy', ('apple,', 937)),
 ('italy', ('citrus', 1449)),
 ('italy', ('sage', 687)),
 ('italy', ('alongside', 4494)),
 ('italy', ('brisk', 281)),
 ('italy', ('acidity.', 2071)),
 ('portugal', ('this', 5504)),
 ('portugal', ('is', 7475)),
 ('portugal', ('ripe', 1831)),
 ('portugal', ('and', 11689)),
 ('portugal', ('fruity,', 307)),
 ('portugal', ('a', 8897)),
 ('portugal', ('wine', 5204)),
 ('portugal', ('that', 2531)),
 ('portugal', ('smooth', 407)),
 ('portugal', ('while', 519)),
 ('portugal', ('still', 650)),
 ('port

(6) rdd4 에 groupByKey를 적용하여 나라별로 (word, count) 튜플을 모은다. 모든 튜플의 리스트는 count의 내림차순으로 정렬하여 그 결과 RDD를 rdd5에 저장한다 (즉, rdd5는 키가 <U>country</U>, 값은 <U>count 내림차순으로 정렬된 (word, count) 튜플의 리스트</U>가 된다) (15점)

힌트: 튜플의 리스트를 second element순으로 정렬하는 방법은 아래 예제코드를 참고하면된다.

````python
sorted([('abc', 121),('abc', 231),('abc', 148), ('abc',221)], key=lambda x: x[1], reverse=True)
````

In [65]:
rdd5 = rdd4.groupByKey().collect()


TypeError: '<' not supported between instances of 'ResultIterable' and 'ResultIterable'

In [60]:
rdd5

[('italy', <pyspark.resultiterable.ResultIterable at 0x7f276ebeb820>),
 ('portugal', <pyspark.resultiterable.ResultIterable at 0x7f276269cd90>),
 ('us', <pyspark.resultiterable.ResultIterable at 0x7f276ec1a5b0>),
 ('spain', <pyspark.resultiterable.ResultIterable at 0x7f276ec1a6d0>),
 ('france', <pyspark.resultiterable.ResultIterable at 0x7f276ec1afa0>),
 ('germany', <pyspark.resultiterable.ResultIterable at 0x7f276ec05070>),
 ('argentina', <pyspark.resultiterable.ResultIterable at 0x7f276ec05fa0>),
 ('chile', <pyspark.resultiterable.ResultIterable at 0x7f276ec051c0>),
 ('australia', <pyspark.resultiterable.ResultIterable at 0x7f276ec05520>),
 ('austria', <pyspark.resultiterable.ResultIterable at 0x7f276ec050d0>),
 ('south africa', <pyspark.resultiterable.ResultIterable at 0x7f276ebeba90>),
 ('new zealand', <pyspark.resultiterable.ResultIterable at 0x7f276ebebe20>),
 ('israel', <pyspark.resultiterable.ResultIterable at 0x7f276ebeb730>),
 ('hungary', <pyspark.resultiterable.ResultIterabl

(7) 이제 나라별로 출현빈도가 top-100에 드는 단어들만 남겨서 rdd6에 저장하도록 하자. rdd6의 키는 country가 되고 값은 단어들의 리스트가 되도록 하자. rdd5에 갖고 있던 단어의 출현빈도는 이제 필요없다. (10점)

(8) 이제 오직 한 나라의 와인 설명에서만 등장하는 단어를 찾기 위해 나라마다의 출현빈도 top-100 단어에 등장하는 단어들이 몇 개 나라의 출현빈도 top-100에 속하는지를 세어보려고 한다. 이를 위해 우선 rdd6에 있는 각 나라별 단어리스트를 flatMap을 이용해 (word, country)의 쌍의 RDD로 변환시킨다. 그 결과는 rdd7이라 이름짓는다. (10점)

(9) rdd7의 각 단어별로 groupByKey한다. 즉, 단어가 key가 되고 단어가 나오는 나라이름의 리스트가 value가 되는 RDD를 생성하여 rdd8이라 명명하시오. (10점)

(10) 마지막으로 오직 하나의 나라에서만 사용되는 단어의 수를 count하여 출력한다. (10점)